In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-fse-selector/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

157 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'system_prompt': 'no-role.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'empty.json'},
  'run': 1},
 'metrics': {}}

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-mini")
    (df["params.dataset.split"] == "train[:16]")
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

117 experiments after preprocessing


In [8]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.temperature,params.run
4,ccot/format-reasoning.txt,cq.txt,ccot-2-shot-thought.json,0.1,"(1, 3)"
8,ccot/format-thought.txt,cq.txt,ccot-2-shot-reasoning.json,0.1,"(1, 2)"
12,ccot/format-thought.txt,cq.txt,ccot-2-shot-thought.json,0.1,"(1, 3)"


In [13]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-reasoning.txt
	ccot/format-thought.txt
	cot/format-minimal.txt
	cot/format-think.txt
	cte/excellent-format-few.txt
	direct/helpful-output-format-few.txt
	direct/minimal-output-format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	ccot-2-shot-reasoning.json
	ccot-2-shot-thought.json
	cot-2-shot.json
	cte-2-shot-delim-1.json
	cte-2-shot-delim-2.json
	direct-2-shot.json
- params.qa.temperature
	0.1
	0.3
	0.5
	0.7
- params.run
	(1, 2, 3)
	(1, 3)
	(1, 2)


In [14]:
df.to_json('results.jsonl', orient='records', lines=True)